In [1]:
import lidar
import rosbag
from pathlib import Path
from dask.distributed import Client
from dask import delayed
import dask.dataframe as dd
import dask

client = Client()  # start distributed scheduler locally.  Launch dashboard
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
bagfile = "/workspaces/lidar/tests/testdata/big.bag"

In [3]:
test0= lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 10)

100%|██████████| 10/10 [00:04<00:00,  2.28it/s]


In [4]:
test1 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 11, end_frame_number = 20)

100%|██████████| 9/9 [00:03<00:00,  2.87it/s]


In [5]:
len(test0)

10

In [6]:
len(test1)

9

In [7]:
test0.extend(test1)

Dataset([Delayed('DataFrame-5a63dda7-74f8-44ae-83ee-c4e45b34648e'), Delayed('DataFrame-fee2f269-587a-42a9-8679-e7c76c987a86'), Delayed('DataFrame-c9c9fab9-6d4e-482c-8d18-56310fcb1e0c'), Delayed('DataFrame-0d9c893d-21e3-4031-b621-44adb176ed5f'), Delayed('DataFrame-1a99c884-63fa-4323-af6b-bda09e2160b5'), Delayed('DataFrame-7ce1e3e4-e9a4-443d-8afe-93edea5dd7d1'), Delayed('DataFrame-7aba969b-8067-4273-99d2-720c60ad6697'), Delayed('DataFrame-f1332f1c-476c-422c-ae8f-4c1a8dd0eb48'), Delayed('DataFrame-9db188b2-f416-4bc7-85a6-f4f67bdb6f1d'), Delayed('DataFrame-19652e93-5f27-4714-979c-fc3d9bd755fc'), Delayed('DataFrame-75e98844-09bd-44f6-a7b8-8d8d2efa89e5'), Delayed('DataFrame-a316b0b3-305c-4947-aef3-6c9f47a7c801'), Delayed('DataFrame-28975417-9f96-495d-b721-46a58bceaac6'), Delayed('DataFrame-e6f12e68-b348-4948-a341-18c4b85ba09c'), Delayed('DataFrame-51c4d377-5592-446b-ba21-2537814f7e3e'), Delayed('DataFrame-f61b7777-280e-4eed-90b3-a67e760adc89'), Delayed('DataFrame-a48cef53-1555-4f2d-b0b8-a1ff

In [8]:

lists = np.array_split(range(10), 3)

NameError: name 'np' is not defined

In [74]:
import numpy as np
max_size = 10
length = 200
test = np.arange(length +1)

result = np.array_split(test, (len(test) // max_size)+1)

In [75]:
result

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39]),
 array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59]),
 array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69]),
 array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]),
 array([80, 81, 82, 83, 84, 85, 86, 87, 88, 89]),
 array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109]),
 array([110, 111, 112, 113, 114, 115, 116, 117, 118, 119]),
 array([120, 121, 122, 123, 124, 125, 126, 127, 128]),
 array([129, 130, 131, 132, 133, 134, 135, 136, 137]),
 array([138, 139, 140, 141, 142, 143, 144, 145, 146]),
 array([147, 148, 149, 150, 151, 152, 153, 154, 155]),
 array([156, 157, 158, 159, 160, 161, 162, 163, 164]),
 array([165, 166, 167, 168, 169, 170, 171, 172, 173]),
 array([174, 175, 176, 177, 178, 179, 180, 181, 182]),
 arra

In [52]:
chunk_num = 0
results = []
for chunk in result:
    print("new")
    print(chunk[0])
    print(chunk[-1])
    dataset = lidar.Dataset.from_file(Path(bagfile), start_frame_number = chunk[0], end_frame_number = chunk[-1])
    print(len(chunk))
    print(dataset)
    results.append(dataset)


new
0
8
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
inside bag 0
inside bag 1
inside bag 2
inside bag 3
inside bag 4
inside bag 5
inside bag 6
inside bag 7
inside bag 8
9
Lidar Dataset with 9 frame(s)
new
9
17
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
in

In [53]:
data = []
[data.extend(dataset.data) for dataset in results]
timestamps = []
[timestamps.extend(dataset.timestamps) for dataset in results]
meta = dataset.meta

In [54]:
joined = lidar.Dataset(data,timestamps,meta)

In [55]:
len(joined)

26

In [76]:
%%time
test25 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 200)

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
inside bag 0
inside bag 1
inside bag 2
inside bag 3
inside bag 4
inside bag 5
inside bag 6
inside bag 7
inside bag 8
inside bag 9
inside bag 10
inside bag 11
inside bag 12
inside bag 13
inside bag 14
inside bag 15
inside bag 16
inside bag 17
inside bag 18
inside bag 19
inside bag 20
inside bag 21
inside bag 22
inside bag 23
inside bag 24
inside bag 25
inside bag 26
inside bag 27
inside bag 28
inside bag 29
inside bag 30
inside bag 31
inside bag 32
inside bag 33
inside bag 34
inside bag 35
inside bag 36
inside bag 37
inside bag 38
inside bag 39
inside bag 40
inside bag 41
inside

In [77]:
len(test25)

201

In [59]:
joined.timestamps == test25.timestamps

True

In [66]:
res = joined.data[20].compute() == test25.data[20].compute()

In [67]:
res.describe()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
count,112257,112257,112257,112257,112257,112257,112257,112257,112257,112257
unique,1,1,1,1,1,1,1,1,1,1
top,True,True,True,True,True,True,True,True,True,True
freq,112257,112257,112257,112257,112257,112257,112257,112257,112257,112257


Ok, so this works, now what about parallel

In [78]:
%%time
chunk_num = 0
results = []
for chunk in result:
    dataset = delayed(lidar.Dataset.from_file)(Path(bagfile), start_frame_number = chunk[0], end_frame_number = chunk[-1])
    results.append(dataset)


CPU times: user 7.67 ms, sys: 4.31 ms, total: 12 ms
Wall time: 9.25 ms


In [72]:
%%time
res = dd.compute(*results)

CPU times: user 1.41 s, sys: 411 ms, total: 1.82 s
Wall time: 15.2 s


In [2]:
t="Dataset([Delayed('DataFrame-a1a9aa96-0827-4cfc-ab3c-0cc2a5bd2f5f'), Delayed('DataFrame-4e4ac984-4651-45f6-876f-2dcb48501c35')],[datetime.datetime(2020, 6, 22, 13, 40, 42, 657267), datetime.datetime(2020, 6, 22, 13, 40, 42, 755912)],{'orig_file': '/workspaces/lidar/tests/testdata/test.bag', 'topic': '/os1_cloud_node/points'})"

In [3]:
len(t)

326